In [5]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

downloads = Path.home() / "Downloads"
counts_path = downloads / "pedestrian-counting-system-monthly-counts-per-hour(1).csv"
out_dir = Path(r"C:\Users\Rohan\OneDrive\Uni\SIT374 - Team Project A\Sprint 2")
out_dir.mkdir(parents=True, exist_ok=True)
output_file = out_dir / "AveragePedestrianTraffic.xlsx"


In [7]:
# Extract lat/lon from Location string
def extract_latlon(s):
    if pd.isna(s):
        return (np.nan, np.nan)
    nums = re.findall(r"-?\d+\.\d+|-?\d+", str(s))
    if len(nums) >= 2:
        try:
            return float(nums[0]), float(nums[1])
        except:
            pass
    return (np.nan, np.nan)

df[["Latitude", "Longitude"]] = df["Location"].apply(lambda x: pd.Series(extract_latlon(x)))
df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

# Timestamp
df["Sensing_Date"] = pd.to_datetime(df["Sensing_Date"], errors="coerce")
df["HourDay"] = pd.to_numeric(df["HourDay"], errors="coerce").fillna(0).astype(int)
df["timestamp"] = df["Sensing_Date"] + pd.to_timedelta(df["HourDay"], unit="h")

# Date Range
start = pd.Timestamp("2023-01-01")
end = pd.Timestamp("2024-11-30 23:59:59")
df = df[(df["timestamp"] >= start) & (df["timestamp"] <= end)].copy()

# Time parts
df["date"] = df["timestamp"].dt.floor("d")
df["hour"] = df["timestamp"].dt.hour
df["weekday"] = df["timestamp"].dt.weekday  # Monday=0


count_col = "Total_of_Directions"
if count_col not in df.columns:
    raise KeyError(f"{count_col} missing.")

In [8]:
daily = (
    df.groupby(["Location_ID", "date", "Location", "Latitude", "Longitude"])[count_col]
    .sum()
    .reset_index(name="daily_pedestrian_count")
)

rush_avg = ((df["hour"].isin([7, 8, 17, 18])) & (df["weekday"] < 5))
rush = (
    df[rush_avg]
    .groupby(["Location_ID", "date"])[count_col]
    .sum()
    .reset_index(name="peak_hour_pedestrian_count")
)

weekend_avg = df["weekday"].isin([5, 6])
weekend = (
    df[weekend_avg]
    .groupby(["Location_ID", "date"])[count_col]
    .sum()
    .reset_index(name="weekend_pedestrian_count")
)

# Merge into sensor date table
sensor_date = daily.merge(rush, on=["Location_ID", "date"], how="left")
sensor_date = sensor_date.merge(weekend, on=["Location_ID", "date"], how="left")
sensor_date[["peak_hour_pedestrian_count", "weekend_pedestrian_count"]] = sensor_date[
    ["peak_hour_pedestrian_count", "weekend_pedestrian_count"]
].fillna(0)

In [9]:
# Export
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    sensor_date.to_excel(writer, sheet_name="SensorDateTraffic", index=False)

display(sensor_date.head(8))
print(f"Exported {len(sensor_date)} rows to {output_file}")

,Location_ID,date,Location,Latitude,Longitude,daily_pedestrian_count,peak_hour_pedestrian_count,weekend_pedestrian_count
0,1,2023-08-01,"-37.81349441, 144.96515323",-37.813494,144.965153,9980,2810.0,0.0
1,1,2023-08-02,"-37.81349441, 144.96515323",-37.813494,144.965153,16405,4283.0,0.0
2,1,2023-08-03,"-37.81349441, 144.96515323",-37.813494,144.965153,12376,3114.0,0.0
3,1,2023-08-04,"-37.81349441, 144.96515323",-37.813494,144.965153,15353,528.0,0.0
4,1,2023-08-05,"-37.81349441, 144.96515323",-37.813494,144.965153,20577,0.0,20577.0
5,1,2023-08-06,"-37.81349441, 144.96515323",-37.813494,144.965153,13096,0.0,13096.0
6,1,2023-08-07,"-37.81349441, 144.96515323",-37.813494,144.965153,11993,3862.0,0.0
7,1,2023-08-08,"-37.81349441, 144.96515323",-37.813494,144.965153,16327,4325.0,0.0


Exported 41068 rows to C:\Users\Rohan\OneDrive\Uni\SIT374 - Team Project A\Sprint 2\sensor_date_level_traffic_with_location.xlsx
